In [1]:
import pandas as pd

In [4]:
# Meta data of the data
meta_data = pd.read_csv('ebu3b/data/ebu3b_metadata.csv')

In [5]:
templates = ['Zone Temperature Sensor', 'Supply Air Flow', 'Damper Position']

sampled_meta_data = meta_data[meta_data['Schema Label'].isin(templates)]

print "Number of files = ", len(sampled_meta_data)

Number of files =  712


In [23]:
df_list = []
for row in sampled_meta_data.iterrows():
    
    # Get the file name
    file_name = "ebu3b/data/" + row[1]['Unique Identifier'] + ".csv"
    
    # Read the csv into pandas data frame
    df = pd.read_csv(file_name)
    
    # filter the non null
    df = df[df["time"].notnull()]

    # convert string to datetime object
    df["time"] = pd.to_datetime(df["time"])
    
    df = df.groupby(df['time'].dt.date)[['value']].count().assign(identifier=row[1]['Unique Identifier'])
    df = df.reset_index().rename(columns={"value" : "count"})

    df_list.append(df)
    if not (len(df_list) % 100):
        print len(df_list)
df_all = pd.concat(df_list)

100
200
300
400
500
600
700


In [27]:
df_stats = df_all.merge(sampled_meta_data, left_on='identifier', right_on="Unique Identifier")

In [103]:
df_stats_small = df_stats[['identifier', 'time', 'count', 'Schema Label', 'Location']]
df_stats_small = df_stats_small.groupby(['time', 'Location', 'Schema Label'])[['count']].sum().unstack().reset_index()
df_stats_small['count', 'total'] = df_stats_small['count'].sum(axis=1)

columns = ['time', 'Location', 'count_damper_position', 'count_supply_flow', 'count_zone_temp', 'count_total']
df_stats_small.columns = columns
df_stats_small = df_stats_small.sort_values(by='count_total', ascending=False)

### how many good pairs are there overall

- Good pairs are those which contain atleast 250 measurements for each signal in a day / room pair

In [108]:
print "Number of good pairs = ",len(df_stats_small[(df_stats_small.count_damper_position > 250) &
                   (df_stats_small.count_supply_flow > 250) & (df_stats_small.count_zone_temp > 250)])

Number of good pairs =  235207


### Top 15 room/day pair

In [109]:
df_stats_small.head(15)

,time,Location,count_damper_position,count_supply_flow,count_zone_temp,count_total
7734,2014-02-02,Rm-3150,524.0,524.0,524.0,1572.0
7731,2014-02-02,Rm-3144,524.0,524.0,524.0,1572.0
7733,2014-02-02,Rm-3148,524.0,524.0,524.0,1572.0
7735,2014-02-02,Rm-3152,524.0,524.0,524.0,1572.0
7736,2014-02-02,Rm-3154,524.0,524.0,524.0,1572.0
7737,2014-02-02,Rm-3200A,524.0,524.0,524.0,1572.0
7738,2014-02-02,Rm-3200B,524.0,524.0,524.0,1572.0
7739,2014-02-02,Rm-3202,524.0,524.0,524.0,1572.0
7740,2014-02-02,Rm-3203,524.0,524.0,524.0,1572.0
7741,2014-02-02,Rm-3205,524.0,524.0,524.0,1572.0


### Top 15 days

In [110]:
df_stats_small.groupby('time').sum().sort_values(by='count_total', ascending=False).head(15)

,count_damper_position,count_supply_flow,count_zone_temp,count_total
time,,,,
2014-02-02,121568.0,122616.0,122616.0,366800.0
2014-02-03,120646.0,121811.0,121914.0,364371.0
2014-02-04,119242.0,120376.0,120052.0,359670.0
2014-02-05,114840.0,115829.0,115820.0,346489.0
2014-06-29,112859.0,114162.0,113472.0,340493.0
2014-02-10,111256.0,112320.0,112320.0,335896.0
2014-02-06,109278.0,110218.0,110489.0,329985.0
2014-03-13,107648.0,108576.0,108714.0,324938.0
2014-05-18,103472.0,104364.0,104364.0,312200.0


### Top 15 weeks

In [111]:
#df_stats_small.groupby().sum()
df_stats_small['time'] = pd.to_datetime(df_stats_small['time'])
df_stats_small.groupby([df_stats_small.time.dt.week, df_stats_small.time.dt.year]).sum().sort_values(
    by='count_total', ascending=False).head(15).reset_index(0).rename(
    columns={"time" : "week"}).reset_index().rename(columns={"time" : "year"}).head(15)

,year,week,count_damper_position,count_supply_flow,count_zone_temp,count_total
0,2014,6,765478.0,772430.0,772241.0,2310149.0
1,2014,21,687600.0,694578.0,691907.0,2074085.0
2,2014,7,668430.0,674104.0,670878.0,2013412.0
3,2016,43,619155.0,621789.0,624424.0,1865368.0
4,2014,1,615516.0,620817.0,622576.0,1858909.0
5,2014,8,611790.0,617292.0,617290.0,1846372.0
6,2016,46,612810.0,615417.0,618025.0,1846252.0
7,2016,41,612715.0,615322.0,617930.0,1845967.0
8,2016,45,612245.0,614851.0,617456.0,1844552.0
9,2014,11,611150.0,616809.0,616119.0,1844078.0
